### Library Import

In [2]:
import joblib

trained_model = joblib.load("../results/HuberRegressor.pkl")

AttributeError: Can't get attribute 'ColumnsGuard' on <module '__main__'>

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

sys.path.append("..")

from src.sanity import *

In [ ]:
np.random.seed(80)
plt.style.use("seaborn-v0_8-darkgrid")
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

%config InlineBackend.figure_format = 'retina'

In [ ]:
from src.load_datasets import load_traffic_data


dataframe = (
    load_traffic_data()
    .pipe(
        select_column,
        cols=[
            "date_time",
            "temp",
            "rain_1h",
            "snow_1h",
            "clouds_all",
            "traffic_volume",
        ],
    )
    .pipe(format_datetime, col="date_time")
    .pipe(set_index, col="date_time")
    .pipe(resample_data, freq="H")
    .pipe(interpolate_column)
)

print(f"dataframe shape {dataframe.shape}")

dataframe.head()

### test_train_split

In [ ]:
from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import temporal_train_test_split

TEST_SIZE = 15
TARGET_COl = "traffic_volume"

train_xy, test_xy = temporal_train_test_split(dataframe, test_size=2 * TEST_SIZE)

print(f"train_xy and test_xy size: {train_xy.shape}, {test_xy.shape}", end="\n\n")

y_train = train_xy[TARGET_COl].copy(deep=True)
x_train = train_xy.drop(columns=[TARGET_COl]).copy(deep=True)

y_test = test_xy[TARGET_COl].copy(deep=True)
x_test = test_xy.drop(columns=[TARGET_COl]).copy(deep=True)

print(f"y_train and x_train size {y_train.shape}, {x_train.shape}")
print(
    f"y_test  and x_test  size {y_test.shape},  {x_test.shape}",
    end="\n\n",
)

plot_series(y_train.tail(62), y_test, labels=["Train", "Test"])

In [ ]:
import joblib

trained_model = joblib.load("../results/HuberRegressor.pkl")

In [ ]:
import joblib

trained_model = joblib.load("../results/HuberRegressor.pkl")


model_n_features_in = trained_model.get_fitted_params()[
    "forecaster__reducer__estimator__n_features_in"
]
model_fname_in = trained_model.get_fitted_params()[
    "forecaster__reducer__estimator__feature_names_in"
]
model_coeff = trained_model.get_fitted_params()["forecaster__reducer__estimator__coef"]
model_intercept = trained_model.get_fitted_params()[
    "forecaster__reducer__estimator__intercept"
]

print(f"model_n_features_in {model_n_features_in}", end="\n\n")
print(f"model_fname_in {model_fname_in}", end="\n\n")
print(f"model_coeff {model_coeff}", end="\n\n")
print(f"model_intercept {model_intercept}", end="\n\n")

In [ ]:
print(x_test.columns)

### Forecasting

In [ ]:
fh = np.arange(1, TEST_SIZE + 1)

y_fcasted = trained_model.predict(fh, X=x_test)

plot_series(y_train.tail(12), y_test, y_fcasted, labels=["Train", "Test", "fcasted"])

In [ ]:
y_fcasted

In [ ]:
print(trained_model.cutoff)

In [ ]:
print(test_xy.shape)
test_xy.head()

In [ ]:
test_xy_new = test_xy.head(TEST_SIZE)
test_xy_future = test_xy.tail(TEST_SIZE)
print(test_xy_new.shape, test_xy_future.shape)

In [ ]:
test_xy_new.tail()

In [ ]:
test_xy_future.head()

In [ ]:
y_new = test_xy_new.pop(TARGET_COl)
test_xy_new.head()

In [ ]:
y_new.head()

In [ ]:
# trained_model.update(y_new,X=test_xy_new,update_params=False)

y_new_fcasted = trained_model.update_predict(
    y_new, X=test_xy_new, update_params=False, reset_forecaster=False
)

In [ ]:
print(trained_model.cutoff)

In [ ]:
y_future = test_xy_future.pop(TARGET_COl)
fh = np.arange(1, TEST_SIZE + 1)

y_future_fcated = trained_model.predict(fh, X=test_xy_future)

In [ ]:
plot_series(
    y_new,
    y_new_fcasted,
    y_future,
    y_future_fcated,
    labels=["y_new", "y_new_fcasted", "y_future", "y_future_fcated"],
)

### No Input Graurd in Fitted Model 

In [ ]:
x_modified = x_test[["press", "wnd_spd", "snow", "rain", "dewp", "temp"]]
x_modified.head()

In [ ]:
fh = np.arange(1, TEST_SIZE + 1)

y_fcasted = trained_model.predict(fh, X=x_modified)

plot_series(y_train.tail(12), y_test, y_fcasted, labels=["Train", "Test", "fcasted"])